##### PyMC3 Examples

# GLM Model Selection using Bayes Factor


**A minimal reproducable example of Model Selection using Bayes Factor.**

+ This example evaluates two different regression models and declares which has the better fit according to a Bayes Factor comparison.
+ The example is adapted specifically from Jake Vanderplas' [recent blogpost](https://jakevdp.github.io/blog/2015/08/07/frequentism-and-bayesianism-5-model-selection/) on model selection, for which he used an emcee sampler and numpy-based likelihood. The main purpose of this Notebook is to create a PyMC3-based version.
+ See also this question on [Cross Validated](http://stats.stackexchange.com/questions/161082/bayesian-model-selection-in-pymc3) 
+ The dataset is tiny and generated within this Notebook. It contains errors in the measured value (y) only.


**Note:**

+ Python 3.4 project using latest available [PyMC3](https://github.com/pymc-devs/pymc3)
+ Developed using [ContinuumIO Anaconda](https://www.continuum.io/downloads) distribution on a Macbook Pro 3GHz i7, 16GB RAM, OSX 10.10.5.  
+ Finally, if runs become unstable or Theano throws weird errors, try clearing the cache `$> theano-cache clear` and rerunning the notebook.


**Package Requirements (shown as a conda-env YAML):**
```
$> less conda_env_pymc3_examples.yml

name: pymc3_examples
    channels:
      - defaults
    dependencies:
      - python=3.4
      - ipython
      - ipython-notebook
      - ipython-qtconsole
      - numpy
      - scipy
      - matplotlib
      - pandas
      - seaborn
      - patsy  
      - pip

$> conda env create --file conda_env_pymc3_examples.yml

$> source activate pymc3_examples

$> pip install --process-dependency-links git+https://github.com/pymc-devs/pymc3

```

# Setup

In [ ]:
%matplotlib inline
%qtconsole --colors=linux

import warnings
warnings.filterwarnings('ignore')

In [ ]:
from collections import OrderedDict
from time import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.optimize import fmin_powell
import pymc3 as pm
import theano as thno
import theano.tensor as T 

from IPython.html.widgets import interactive, fixed

# configure some basic options
sns.set(style="darkgrid", palette="muted")
pd.set_option('display.notebook_repr_html', True)
plt.rcParams['figure.figsize'] = 12, 8
rndst = np.random.RandomState(0)

## Local Functions

In [ ]:
def generate_data(n=20, p=0, a=1, b=1, c=0, latent_sigma_y=20):
    ''' 
    Create an example dataset based on a very simple model that we might
    imagine is a noisy physical process:
        1. random x values within a range
        2. latent error aka inherent noise in y
        3. optional outliers from unknown sources

    Model form: y ~ a + bx + cx^2 + e
    
    NOTE: latent_sigma_y is used to create a normally distributed,
    'latent error' aka 'inherent noise' in the 'physical process' 
    generating thses values, rather than experimental measurement error. 
    Please don't use the returned `latent_error` values in inferential 
    models, it's there for interest only.
    '''
    
    df = pd.DataFrame({'x':rndst.randint(0,100,n)})
    df['latent_error'] = rndst.normal(0,latent_sigma_y,n)
    df['outlier'] = rndst.binomial(1,p,n)

    bimdl = np.append(rndst.normal(-.4,0.1,100), np.random.normal(.4,0.1,100))
    df['outlier_adj'] = rndst.choice(bimdl, n, replace=False)
                
    ## create linear model
    df['y'] = a + b*(df['x']) + c*(df['x'])**2 + df['latent_error']
   
    ## add extreme noise for outliers
    df['y'] = df['y'] + (df['outlier'] * df['outlier_adj'] * df['y'])

    ## round and return
    for col in ['y','latent_error','x','outlier_adj']:
        df[col] = np.round(df[col],3)
       
    return df


def sketch_data(n=20, p=0, a=-30, b=5, c=0, latent_sigma_y=20):
    ''' Sketch the generated data '''
    
    df = generate_data(n, p, a, b, c, latent_sigma_y)
    ordr = 'linear' if c == 0 else 'quadratic'
    
    g = sns.FacetGrid(df, size=8, hue='outlier', hue_order=[True,False],
                  palette='Set1', legend_out=False)

    _ = g.map(plt.errorbar, 'x', 'y', 'latent_error', marker="o"
              , ls='', elinewidth=0.7).add_legend()

    plotx = np.linspace(df['x'].min() - np.ptp(df['x'])*.1
                        ,df['x'].max() + np.ptp(df['x'])*.1, 100)
    ploty = a + b*plotx + c*plotx**2

    _ = plt.plot(plotx,ploty,'--',alpha=0.8)
    plt.subplots_adjust(top=0.92)
    _ = g.fig.suptitle('Sketch of Data Generation ({})'.format(ordr)
                       ,fontsize=16)
    

def plot_traces(traces, retain=1000):
    ''' Convenience function for plotting traces with overlaid means and values
    '''
    
    ax = pm.traceplot(traces[-retain:], figsize=(12,len(traces.varnames)*1.5),
        lines={k: v['mean'] for k, v in pm.df_summary(traces[-retain:]).iterrows()})

    for i, mn in enumerate(pm.df_summary(traces[-kp:])['mean']):
        ax[i,0].annotate('{:.2f}'.format(mn), xy=(mn,0), xycoords='data'
                    ,xytext=(5,10), textcoords='offset points', rotation=90
                    ,va='bottom', fontsize='large', color='#AA0022')

    
def create_poly_modelspec(k=1):
    ''' Convenience function to create a polynomial modelspec string 
    '''
    return ('y ~ 1 + x ' + ' '.join(['+ np.power(x,{})'.format(j) for j in range(2,k+1)])).strip()

## Draft Data

We'll generate a dummy dataset based on a specific distribution, so we can better evaluate the correctness of the automated model selection later.

Use the interactive plot below to get a feel for the possibilities of data we could generate.

In [ ]:
interactive(sketch_data, n=[5,50,5], p=[0,.5,.05], a=[-50,50], b=[-10,10], c=[-10,10]
            ,latent_sigma_y=[0,100,5])

**Observe:**

+ I've shown the `latent_error` in errorbars, but this is for interest only, since this shows the _inherent noise_ in whatever 'physical process' we imagine created the data.
+ There is no _measurement error_.
+ Datapoints created as outliers are shown in **red**, again for interest only.

## Create Dataset for Modelling

We can use the above interactive plot to get a feel for the effect of the params. Now I'll create a fixed dataset to use for the remainder of the Notebook. 

For a start, I'll create a linear model with no outliers. Keep it simple.

In [ ]:
df = generate_data(n=10, p=0, a=-30, b=5, latent_sigma_y=60)

In [ ]:
g = sns.lmplot(x='x', y='y', data=df, fit_reg=False, size=8
               ,scatter_kws={'alpha':0.7,'s':100})

**Observe:**

+ This then, is our dataset for the remainder of the notebook. Let's see how well we can fit it!

### Standardize

In [ ]:
dfs = df.copy()
dfs['x'] = (df['x'] - df['x'].mean()) / df['x'].std()

In [ ]:
## create ranges for later ylim xim
dfs_ylims = (dfs['y'].min() - np.ptp(dfs['y'])/10,dfs['y'].max() + np.ptp(dfs['y'])/10)
dfs_xlims = (dfs['x'].min() - np.ptp(dfs['x'])/10,dfs['x'].max() + np.ptp(dfs['x'])/10)

---

# Create Simple OLS Model

This *linear model* is really simple and conventional:
$$y = a + bx + \epsilon$$



## Define model using ordinary pymc3 method

In [ ]:
with pm.Model() as mdl_ols:
        
    ## define Normal priors to give Ridge regression
    b0 = pm.Normal('b0', mu=0, sd=100)
    b1 = pm.Normal('b1', mu=0, sd=100)
 
    ## define Linear model
    yest = b0 + b1 * df['x']

    ## define Normal likelihood with HalfCauchy noise (fat tails, equiv HalfT 1DoF)
    sigma_y = pm.HalfCauchy('sigma_y', beta=10)
    likelihood = pm.Normal('likelihood', mu=yest, sd=sigma_y, observed=df['y'])

    ## sample using NUTS (starting from MAP found using powell)
    start_MAP = pm.find_MAP(fmin=fmin_powell, disp=True)
    traces_ols = pm.sample(2000, start=start_MAP, step=pm.NUTS(), progressbar=True)

##### View Traces after burn-in

In [ ]:
plot_traces(traces_ols, retain=1000)

**Observe:**

+ This simple OLS manages to make fairly good guesses - the data has been generated fairly simply after all - but it does appear to have been fooled by the inherent noise and outliers.


## Define model using pymc3 GLM method

PyMC3 has a quite recently developed method - `glm` - for defining models using a `patsy`-style formula syntax. This seems really useful, especially for defining simple regression models in fewer lines of code. 

I couldn't find a direct comparison in the the examples, so before I launch into using `glm` for the rest of the Notebook, here's the same OLS model as above, defined using `glm`.

In [ ]:
with pm.Model() as mdl_ols_glm:

    # setup model with Normal likelihood (which uses HalfCauchy for error prior)
    pm.glm.glm('y ~ 1 + x', df, family=pm.glm.families.Normal())
    
    ## sample using NUTS (starting from MAP found using powell)
    start_MAP = pm.find_MAP(fmin=fmin_powell, disp=True)
    traces_ols_glm = pm.sample(2000, start=start_MAP, step=pm.NUTS(), progressbar=True)

##### View Traces after burn-in

In [ ]:
plot_traces(traces_ols_glm, retain=1000)

**Observe:**

+ The output parameters are of course named differently to my custom naming before. Now we have:

    `b0 == Intercept`  
    `b1 == x`  
    `sigma_y_log == sd_log`  
    `sigma_y == sd`  
    
    
+ However, naming aside, this `glm`-defined model appears to behave in a very similar way, and find pretty much the same parameter values as the conventionally-defined model - any differences are due to the random nature of the sampling.
+ I'll quite happily use the `glm` syntax for further models below.

---

# Create Higher-Order OLS Models

Back to the real purpose of this Notebook: demonstrate model selection using Bayes Factor

In [ ]:
models, traces = OrderedDict(), OrderedDict()

for k in range(1,8):

    nm = 'k{}'.format(k)
    fml = create_poly_modelspec(k)
    
    with pm.Model() as models[nm]:

        print('\nRunning: {}'.format(nm))
        # setup model with Normal likelihood (which uses HalfCauchy for error prior)
        pm.glm.glm(fml, dfs, family=pm.glm.families.Normal())

        ## sample using NUTS (starting from MAP found using powell)
        start_MAP = pm.find_MAP(fmin=fmin_powell, disp=False)
        traces[nm] = pm.sample(2000, start=start_MAP, step=pm.NUTS(), progressbar=True)    

##### Compare likelihoods

In [ ]:
## get negative log likelioods straight from model.logp

loglks = OrderedDict()

for nm, mdl in models.items():
    loglks[nm] = -mdl.logp(pm.df_summary(traces[nm])['mean'].to_dict())

dfll = pd.DataFrame.from_dict(loglks, orient='index')
dfll.rename(columns={0:'loglikelihood'}, inplace=True)
ax = dfll.plot(kind='bar', legend=False)

In [ ]:
plot_traces(traces['k2'])

### View Posterior Predictive Fit

In [ ]:
g = sns.lmplot(x='x', y='y', data=dfs, fit_reg=False, size=8
               ,scatter_kws={'alpha':0.7,'s':100})

def lm(x, s):
    return s['Intercept'] \
            + s['x']*x \
            + s['np.power(x, 2)']*np.power(x,2) \
            + s['np.power(x, 3)']*np.power(x,3) \
            + s['np.power(x, 4)']*np.power(x,4) \
            + s['np.power(x, 5)']*np.power(x,5)

pm.glm.plot_posterior_predictive(traces['k5'][-1000:]
        ,eval=np.linspace(dfs_xlims[0], dfs_xlims[1], 200)
        ,lm=lm, samples=500, color='#00AA00', alpha=.2)

_ = g.axes[0][0].set_ylim(dfs_ylims)
_ = g.axes[0][0].set_xlim(dfs_xlims)

In [ ]:
g = sns.FacetGrid(df, size=8, hue='outlier', hue_order=[True,False],
                  palette='Set1', legend_out=False)

lm = lambda x, samp: samp['b0'] + samp['b1'] * x
xrng = df['x'].max() - df['x'].min()

pm.glm.plot_posterior_predictive(traces_ols[-1000:]
        ,eval=np.linspace(df['x'].min() - np.ptp(df['x'])/10
                        ,df['x'].max() + np.ptp(df['x'])/10, 10)
        ,lm=lm, samples=200, color='#00AA00', alpha=.1)

_ = g.map(plt.errorbar, 'x', 'y', 'sigma_y', marker="o", ls='').add_legend()


plotx = np.linspace(df['x'].min() - np.ptp(df['x'])/10
                        ,df['x'].max() + np.ptp(df['x'])/10, 10)
ploty = -30 + 5* plotx

_ = plt.plot(plotx,ploty)

In [ ]:
df['sigma_y'].plot(kind='kde')

---

---

# next

---

Example originally contributed by Jonathan Sedar 2015-12-30 [github.com/jonsedar](https://github.com/jonsedar)